In [ ]:
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
import numpy as np
import ast

In [ ]:
# Dresden  Station ID = 1048 , Stations-Kennziffer 10488 (Klotzsche)
# Freiburg: Station ID - 1443, 10803  - ? 
 
# 10 minute data https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/